In [24]:
import pandas as pd

import sqlite3

import xgboost as xgb
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [6]:
cbb21 = pd.read_csv('CSV_Files/cbb21_top_68_clean.csv', index_col = 0)
cbb_teams = pd.read_csv('CSV_Files/cbb_teams.csv', index_col = 0)

In [7]:
cbb21

,TEAM,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,Michigan,118.1,91.1,0.9521,54.9,44.9,16.3,15.1,29.4,24.8,28.9,24.5,53.3,42.3,38.7,33.5,66.9,7.2,1.0
1,Baylor,123.2,94.5,0.9548,57.5,49.1,17.6,24.6,37.5,30.9,27.0,31.7,54.1,48.1,41.8,34.0,68.8,6.6,1.0
2,Illinois,117.7,90.4,0.9539,55.6,46.6,18.2,16.1,33.0,22.2,39.2,30.5,55.3,45.4,37.6,32.7,70.7,8.9,1.0
3,Gonzaga,125.4,89.8,0.9791,61.0,47.5,16.1,20.3,30.4,23.4,36.7,25.9,64.0,46.8,36.5,32.5,74.6,8.5,1.0
4,Iowa,123.5,95.7,0.9491,54.6,48.3,13.3,16.3,30.7,28.6,32.0,22.6,52.4,45.8,38.6,34.8,70.0,5.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Mount St. Mary's,95.7,100.6,0.3593,48.7,45.5,20.1,16.3,31.7,24.6,29.7,24.6,47.7,45.6,33.7,30.1,62.2,-7.2,16.0
64,Hartford,97.3,99.6,0.4345,50.4,47.6,19.0,20.4,24.8,28.6,28.6,25.0,51.3,50.7,32.7,28.7,66.9,-4.9,16.0
65,Norfolk St.,99.4,104.1,0.3698,49.7,47.4,17.8,20.7,28.8,28.7,38.6,42.0,46.1,47.2,37.4,31.7,67.9,-5.0,16.0
66,Texas Southern,97.3,103.5,0.3307,48.3,46.5,21.4,19.2,32.9,25.7,38.9,33.8,51.0,45.2,27.7,32.4,71.8,-4.5,16.0


In [9]:
cbb_teams

,TEAM,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,6,1.0,2016
1,Wisconsin,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,6,1.0,2015
2,Michigan,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,6,3.0,2018
3,Texas Tech,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,6,3.0,2019
4,Gonzaga,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,6,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,Michigan St.,111.4,87.8,0.9392,50.6,44.5,20.8,19.2,36.1,27.6,...,32.4,50.4,44.3,34.1,30.1,64.4,6.7,3,3.0,2013
444,Arizona,114.4,92.2,0.9229,52.5,46.6,19.5,19.8,35.0,26.7,...,32.9,50.6,43.4,37.1,35.8,66.8,4.6,3,6.0,2013
445,Oregon,104.8,88.6,0.8728,49.3,46.4,21.4,22.0,35.8,27.2,...,33.3,49.1,44.9,33.3,33.4,69.2,2.9,3,12.0,2013
446,La Salle,112.0,96.2,0.8516,51.9,49.3,17.1,21.3,29.0,34.2,...,28.5,49.3,50.6,37.7,30.2,66.0,0.3,3,13.0,2013


In [10]:
Postseason_modeling = {
    1: 'Round_of_64',
    2: 'Round_of_64',
    3: 'Sweet_16',
    4: 'Elite_8',
    5: 'Final_4',
    6: '2nd_Place',
    7: 'Champion',
}

cbb_teams['POSTSEASON'] = cbb_teams['POSTSEASON'].replace(Postseason_modeling)

In [13]:
cbb_teams[cbb_teams.POSTSEASON == 'Champion']

,TEAM,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
7,Duke,125.2,90.6,0.9764,56.6,46.5,16.3,18.6,35.8,30.2,...,23.9,55.9,46.3,38.7,31.4,66.4,10.7,Champion,1.0,2015
8,Virginia,123.0,89.9,0.9736,55.2,44.7,14.7,17.5,30.4,25.4,...,26.3,52.5,45.7,39.5,28.9,60.7,11.1,Champion,1.0,2019
9,North Carolina,121.0,91.5,0.9615,51.7,48.1,16.2,18.6,41.3,25.0,...,31.6,51.0,46.3,35.5,33.9,72.8,8.4,Champion,1.0,2017
10,Villanova,123.1,90.9,0.9703,56.1,46.7,16.3,20.6,28.2,29.4,...,30.0,57.4,44.1,36.2,33.9,66.7,8.9,Champion,2.0,2016
11,Villanova,128.4,94.1,0.9725,59.5,48.5,15.0,18.2,29.6,27.1,...,26.7,59.0,49.0,40.1,31.7,69.6,10.6,Champion,1.0,2018
12,Connecticut,112.5,91.3,0.9171,51.5,44.6,17.3,19.6,30.4,32.8,...,35.7,48.1,42.2,38.7,33.0,64.8,4.7,Champion,7.0,2014
13,Louisville,115.9,84.5,0.9743,50.6,44.8,18.3,27.0,38.2,33.3,...,34.9,50.8,43.4,33.3,31.8,67.1,9.0,Champion,1.0,2013


In [14]:
cbb_teams.columns

Index(['TEAM', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR', 'TORD',
       'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O', '3P_D', 'ADJ_T',
       'WAB', 'POSTSEASON', 'SEED', 'YEAR'],
      dtype='object')

In [17]:
cbb_teams = cbb_teams.drop(columns = ['BARTHAG', 'YEAR'])

## Base Model

Using just CBB teams to get a basic model going. This will be a basis to see how well the model does using just the xgboost model. Will later improve upon this to better handle the disparity in postseason

In [25]:
X = cbb_teams.drop(columns = ['TEAM', 'POSTSEASON'])
y = cbb_teams.POSTSEASON

In [26]:
# train test split for 5 class data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 24, test_size = .25) 

In [27]:
xgb_clf = xgb.XGBClassifier()

In [28]:
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
# Getting predictions on the testing data then finding the accuracy of those predictions

xgb_pred = xgb_clf.predict(X_test)

accuracy_score(y_test, xgb_pred)

0.6785714285714286